# Recipe Parse

## Define Corpora

In [1]:
import nltk
from nltk.corpus import wordnet as wn
lemma = nltk.wordnet.WordNetLemmatizer()
def create_ingredient_corpus():
    food1 = wn.synset('food.n.01')
    food2 = wn.synset('food.n.02')
    food3 = wn.synset('food.n.03')
    f1 = list(set([w.lower() for s in food1.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    f2 = list(set([w.lower() for s in food2.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    f3 = list(set([w.lower() for s in food3.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    ings = list(set(f1 + f2 + f3))
    for i in range(len(ings)):
        ings[i] = lemma.lemmatize(ings[i])
    return(list(set(ings)))
ingredient_corpus = create_ingredient_corpus()
len(set(ingredient_corpus))

3465

In [2]:
unit_corpus = list(set(['oz', 'ounces', 'ounce', 'gram', 'grams', 'ml', 'l', 'pound', 'lb',
        'ozs', 'stone', 'st', 's.t.', 'milliliters', 'ton', 't', 'micrograms',
        'microgram', 'kilograms', 'kg', 'kilogram', 'metric ton', 'mt', 'm.t',
        'metric tonne', 'cubic inch', 'cubic inches', 'cubic feet', 'f3', 'in3',
        'liter', 'cubic meter', 'm3', 'tsp', 'teaspoons', 'teaspoon', 'tbsp',
        'tablespoons', 'cup', 'cups', 'c', 'floz', 'fluid oz', 'fluid ounces', 
        'quart', 'qu', 'qt', 'pint', 'pt', 'gallons', 'gal', 'tablespoon', 'tablespoons',
                       'pinch', 'as needed']))

In [3]:
def add_to_corpus(ingredient, c):
    ingredient = ingredient.split()
    for i in range(len(ingredient)):
        ingredient[i] = lemma.lemmatize(ingredient[i].lower().strip())
    ingredient = '_'.join(ingredient)
    return(list(set(c + [ingredient])))
ingredient_corpus = add_to_corpus('canola oil', ingredient_corpus)
ingredient_corpus = add_to_corpus('olive oil', ingredient_corpus)
ingredient_corpus = add_to_corpus('garam masala', ingredient_corpus)
ingredient_corpus = add_to_corpus('vegetable oil', ingredient_corpus)

## Sample Recipe Scrape

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np

In [5]:
def example_recipe_scrape(num):
    """
    Sample code to produce a list of ingredients for a given recipe.
    The example recipe is 'Whole-wheat-pancakes-from-scratch' from the 'Pancakes' subcategory under the 'Breakfast and Brunch' category.
    :return: list of string ingredients. These need to be parsed.
    """
    example_sub_category = 'https://www.allrecipes.com/recipes/' + num + '/breakfast-and-brunch-pancakes/'
    page = requests.get(example_sub_category)
    soup = BeautifulSoup(page.content)
    grid = soup.find('div', attrs={'class': 'fixed-grid'})
    article = grid.find('article', attrs={'class': 'fixed-recipe-card'})
    example_recipe_url = article.find('a').get('href')
    recipe_request = requests.get(example_recipe_url)
    recipe_soup = BeautifulSoup(recipe_request.content)
    ingredients_space = recipe_soup.find('div', attrs={'id': 'polaris-app'})
    ingredients_soup = ingredients_space.find_all('label', attrs={'ng-class': '{true: \'checkList__item\'}[true]'})
    ingredients = []
    for ingredient in range(len(ingredients_soup)):
        ingredients.append(ingredients_soup[ingredient]['title'])

    return ingredients

In [6]:
example_recipe_scrape(str(152))

C:\Users\natha\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\natha\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['8 Yukon Gold potatoes, quartered',
 '1 tablespoon dried rosemary',
 '1/4 cup olive oil',
 'salt and pepper to taste']

In [47]:
def remove_parens(string):
    while True:
        if '(' in string:
            left = string.index('(')
            right = string.index(')')
            if left > 0:
                if right == len(string) - 1:
                    string = string[:left] # there is nothing to the right of the right parenthesis
                elif right < len(string) - 2:
                    string = string[:left] + string[right+2:] # get rid of underscore after right parenthesis
                elif right < len(string) - 1:
                    string = string[:left] + string[right+1:]
            elif left == 0:
                string = string[right+1:] # there is nothing to the left of the left parenthesis
        elif ')' in string: # only needed when there is a set of parentheses within another
            ind = string.index(')')
            if ind == len(string) - 1:
                string = string[:ind]
            else:
                string = string[:ind] + string[ind+1:]
        else:
            break
        string = string.strip()
    return(string)

In [49]:
remove_parens('2 (10.75 ounce) cans condensed cream of potato soup')

'2 cans condensed cream of potato soup'

In [60]:
def preprocess_ingredient(string):
    string = string.lower() # lower case
    if ',' in string:
        comma = string.index(',')
        string = string[0:comma] # truncate column name to before the comma (assuming directions to the right)
    return(string)

In [107]:
import UnitConversion as uc

"""
@param recipe - an array where each entry is as follow: (measurement, unit, ingredient)

Extract the measurement and unit of the ingredient as well as whether or not it should be expressed as a quantity
An ingredient, for our purposes, is expressed as EITHER a measurement (e.g. 3 referring to cups) OR a quantity 
(e.g. 3 referring to eggs)
Quantity will be set to True or False based on these observations
"""
def parse_recipe(recipe, dictionary = {}, col_list = [], measurement_list = [], iteration = 0):
    ings = len(recipe)
    for i in range(ings):
        # remove parentheses from ingredients
        recipe[i] = remove_parens(recipe[i])
        recipe[i] = preprocess_ingredient(recipe[i])
        cur = recipe[i].split() # make the ingredient into an array itself!
        # lemmatize the ingredient specification
        for j in range(len(cur)):
            cur[j] = lemma.lemmatize(cur[j])
            if ',' in cur[j]:
                comma = cur[j].index(',')
                word = cur[j][:comma]
        ind = 0
        if cur[ind].lower().strip() == 'about': # if the first word is 'about', check the next word for the measurement
            ind += 1
            
        first = cur[ind]
        unit = cur[ind+1]
        ingred = '_'.join(cur[ind+2:]) # the rest of the entry is the ingredient specification joint by underscore
        
        # check if measurement is in fraction form
        if '/' in first:
            frac = first.split('/') # split numerator from denominator
            num, denom = (frac[0], frac[1])
            # make sure / isn't just the 'or' symbol
            try:
                num = int(num)
                denom = int(denom)
                measurement = num / denom
                quantity = False
            except: # if it is, default the measurement to 1 'quantity'
                measurement = 1
                quantity = True
        elif '/' in unit:
            ind += 1
            unit = cur[ind+1]
            ingred = '_'.join(cur[ind+2:])
            frac = cur[ind].split('/')
            try:
                num, denom = (frac[0], frac[1])
                num = int(num)
                denom = int(denom)
                measurement = int(first) + num / denom
                quantity = False
            except:
                measurement = 1
                quantity = True
        else:
            # Check if the first word of the ingredient is even a number at all
            try:
                measurement = int(first)
                if first == cur[ind]: # in the case that the first word is 'about'
                    unit = cur[ind+1]
                    ingred = '_'.join(cur[ind+2:])
                    quantity = False
                else:
                    unit = cur[1]
                    ingred = '_'.join(cur[ind+1:])
                    quantity = False
            except:
                # in this case, there is not measurement for this ingredient
                # it is itself a unit, e.g. cooking spray
                measurement = 1
                quantity = True
                unit = ''
                ingred = '_'.join(cur)
                        
        # If quantity is False, we do a unit conversion of the found measurement. However, if the found "unit"
        # (second or third word) does not match a unit in our dictionary, we express the ingredient as a quantty
        # This is checked in the convert function itself
        # We won't worry about making a dataframe just yet. We will first list all the columns and the corresponding values
        # unit = re.sub(r'[^\w\s]','', unit)
        if quantity == True:
            col_list.append(ingred)
            measurement_list.append(measurement)
        elif quantity == False:
            if unit not in unit_corpus:
                print(unit, 'not in unit_corpus')
                if ingred != '':
                    ingred = '_'.join([unit] + [ingred])
                else:
                    ingred = unit
                quantity = True
            else:
                mes, cols, quantity = uc.convert(measurement, ' '.join(cur), ingred)
            if quantity == True:
                col_list.append(ingred)
                measurement_list.append(measurement)
            elif quantity == False: # if we know for sure the unit we found is in our dictionary, we have a valid measurement
                col_list.extend(cols)
                measurement_list.extend(mes)
    # lemmatize
    for i in range(len(col_list)):
        temp = col_list[i].split('_')
        for j in range(len(temp)):
            temp[j] = lemma.lemmatize(temp[j])
        col_list[i] = '_'.join(temp)
#     print('\n Column List:', col_list)
#     print('\n Measurement List:', measurement_list)
    
    # Now populate the dictionary
    # Either add a value to the list of values corresponding to a key 
    # Or create a new key value pair
    # init key value pairs as col_list and empty lists per column
    if len(dictionary) == 0:
        dictionary = dict(zip(col_list, [[] for i in range(len(col_list))]))
#     print(dictionary)
    for i in set(col_list + list(dictionary.keys())):
        if i in col_list:
            if i in dictionary:
                dictionary[i] += [measurement_list[col_list.index(i)]]
            else:
                dictionary[i] = [0]*iteration + [measurement_list[col_list.index(i)]]
        else:
            if i in dictionary:
                dictionary[i] += [0]
            else:
                dictionary[i] = [0]
    return(dictionary) # will be used to construct dataframe

In [108]:
d = parse_recipe(['1/2 cup wheat germ',
 '2 cups whole wheat flour',
 '1 teaspoon baking soda',
 '1/2 teaspoon salt',
 '3 cups buttermilk',
 '2 eggs, lightly beaten',
 '1 tablespoon canola oil',
 'cooking spray'], dictionary = {}, col_list = [], measurement_list = [],iteration = 1)
print('\n', d)

egg not in unit_corpus

 {'wheat_germ_mass': [0], 'wheat_germ_volume': [120.0], 'whole_wheat_flour_mass': [0], 'whole_wheat_flour_volume': [480], 'baking_soda_mass': [0], 'baking_soda_volume': [14.787], 'salt_mass': [0], 'salt_volume': [7.393], 'buttermilk_mass': [0], 'buttermilk_volume': [720], 'egg': [2], 'canola_oil_mass': [0], 'canola_oil_volume': [14.787], 'cooking_spray': [1]}


In [109]:
d = {}
for j in range(5):
    rec = example_recipe_scrape(str(j + 147))
    print(rec)
    d = parse_recipe(rec, dictionary = d, col_list = [], measurement_list = [], iteration = j)
    print('\n')

C:\Users\natha\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\natha\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['1 cup all-purpose flour', '1 cup milk', '1 egg', '1 pinch salt']
egg not in unit_corpus


['1 (8 ounce) package vegetarian sausage links (such as Morningstar Farms(R))', '1/2 cup water, or more as needed', '4 cups shredded sweet potatoes', '1/4 cup butter, melted', '1 1/2 (8 ounce) packages shredded, reduced-fat mild Cheddar-mozzarella cheese blend', '1/2 cup finely chopped onion', '1 cup finely sliced fresh spinach leaves', '1 (16 ounce) container low-fat small curd cottage cheese', '8 jumbo eggs']
package not in unit_corpus
package not in unit_corpus
container not in unit_corpus
jumbo not in unit_corpus


['4 ripe bananas, mashed', '8 slices French bread', '1/2 cup milk', '2 eggs', '1 teaspoon vanilla extract', '1/2 teaspoon ground cinnamon']
ripe not in unit_corpus
slice not in unit_corpus
egg not in unit_corpus


['1 (32 ounce) package frozen hash brown potatoes', '8 ounces cooked, diced ham', '2 (10.75 ounce) cans condensed cream of potato soup', '1 (16 ounce) container sour cre

In [110]:
df = pd.DataFrame(data = np.array(list(d.values())).T, columns = d.keys())
list(df.columns)

['all-purpose_flour_mass',
 'all-purpose_flour_volume',
 'milk_mass',
 'milk_volume',
 'egg',
 'salt_mass',
 'salt_volume',
 'shredded_sweet_potato_volume',
 'butter_volume',
 'package_vegetarian_sausage_link',
 'jumbo_egg',
 'finely_chopped_onion_mass',
 'water_volume',
 'butter_mass',
 'container_low-fat_small_curd_cottage_cheese',
 'finely_chopped_onion_volume',
 'finely_sliced_fresh_spinach_leaf_mass',
 'package_shredded',
 'finely_sliced_fresh_spinach_leaf_volume',
 'shredded_sweet_potato_mass',
 'water_mass',
 'slice_french_bread',
 'vanilla_extract_mass',
 'vanilla_extract_volume',
 'ground_cinnamon_volume',
 'ripe_banana',
 'ground_cinnamon_mass',
 'grated_parmesan_cheese_volume',
 'grated_parmesan_cheese_mass',
 'shredded_sharp_cheddar_cheese_volume',
 'container_sour_cream',
 'package_frozen_hash_brown_potato',
 'cooked_mass',
 'can_condensed_cream_of_potato_soup',
 'shredded_sharp_cheddar_cheese_mass',
 'cooked_volume',
 'baking_powder_mass',
 'whole_wheat_flour_volume',
 'w

In [114]:
df['cooked_volume'] # expects everything after comma to be irrelevant --- one case of this not happening (could fix manually)

0      0.000
1      0.000
2      0.000
3    118.294
4      0.000
Name: cooked_volume, dtype: float64

In [115]:
df

,all-purpose_flour_mass,all-purpose_flour_volume,milk_mass,milk_volume,egg,salt_mass,salt_volume,shredded_sweet_potato_volume,butter_volume,package_vegetarian_sausage_link,...,shredded_sharp_cheddar_cheese_mass,cooked_volume,baking_powder_mass,whole_wheat_flour_volume,whole_wheat_flour_mass,baking_powder_volume,blueberry_volume,blueberry_mass,artificial_sweetener_volume,artificial_sweetener_mass
0,0.0,240.0,0.0,240.0,1.0,0.0,14.787,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,960.0,60.0,1.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0
2,0.0,0.0,0.0,120.0,2.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,118.294,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0
4,0.0,0.0,0.0,240.0,1.0,0.0,7.393,0.0,0.0,0.0,...,0.0,0.000,0.0,300.0,0.0,29.574,120.0,0.0,14.787,0.0
